# GenAI: Email Personalisation

In [2]:
path = 'C:\\Users\\esteb\\Documents\\GitHub\\customer-churn-and-personalisation\\data\\loyalty_profiles.csv'
llm_model = 'gemini-1.5-pro-latest'
high_risk_prob = 0.5

In [3]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown
from email.message import EmailMessage

In [4]:
load_dotenv()

try:
    SENDER_EMAIL = os.environ["EMAIL_SENDER"]
    SENDER_PASSWORD = os.environ["EMAIL_PASSWORD"]
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Environment variables loaded successfully.")
except KeyError as e:
    print(f"Error: Environment variable {e} not found.")
    print("Please ensure you have a .env file with EMAIL_SENDER, EMAIL_PASSWORD, and GEMINI_API_KEY.")
    exit()


# --- Load the Customer Profiles ---
try:
    profiles_df = pd.read_csv(path)
    print("Loyalty profiles loaded successfully.")
    # Filter for only the high-risk customers we want to target
    high_risk_customers = profiles_df[profiles_df['churn_probability'] > high_risk_prob].copy()
    print(f"Found {len(high_risk_customers)} high-risk customers to target.")
except FileNotFoundError:
    print("Error: loyalty_profiles.csv not found. Please ensure the file is in the correct directory.")
    high_risk_customers = pd.DataFrame() # Create an empty df to avoid errors later

Environment variables loaded successfully.
Loyalty profiles loaded successfully.
Found 82 high-risk customers to target.


In [ ]:
MASTER_PROMPT = """
You are an expert marketing copywriter for Vodafone, a top-tier telecommunications company.
Your task is to write a personalized, persuasive, and friendly email to a valued customer who our data suggests might be at risk of leaving. The goal is to proactively address their potential concerns and retain them with a relevant offer, making them feel appreciated and understood.

### STRICT GUIDELINES TO FOLLOW ###

**1. Tone Attributes:**
   - **Friendly and Approachable:** Use warm, conversational language. Avoid technical jargon.
   - **Clear and Concise:** Messages must be straightforward and easy to digest. Use short sentences.
   - **Positive and Reassuring:** Highlight benefits and positive outcomes. Address concerns empathetically.
   - **Professional and Trustworthy:** Maintain a respectful tone. Ensure information is reliable.

**2. Email Structure:**
   - **1. Subject Line:** Must be friendly, enticing, and personalized. (e.g., "Exclusive Benefits Await You, {customer_name}!")
   - **2. Greeting:** Must be "Hi {customer_name},"
   - **3. Introduction:** Briefly and warmly explain the purpose of the email.
   - **4. Body:** Highlight the special, relevant offer. Use bullet points for key information.
   - **5. Call to Action (CTA):** Must be clear and compelling. (e.g., "Click here to explore your exclusive benefits.")
   - **6. Closing:** Must be warm and appreciative.
   - **7. Signature:** Must be "Best regards,\nThe Vodafone Customer Care Team"

**3. Constraints:**
   - **DO NOT** mention the words "churn," "risk," "leaving," "canceling," or "unhappy." The tone must be entirely positive and proactive.
   - **DO NOT** invent specific prices or discount percentages unless they are provided in the {suggested_offer}.
   - **DO NOT** make up features or services that don't exist.

### CUSTOMER DATA & TASK ###

You will write an email for the following customer:

- **Customer Name:** {customer_name}
- **Customer Since:** {tenure_months} months
- **Loyalty Profile:** {loyalty_profile}
- **Primary Churn Driver (Internal Use Only, do not mention directly):** {primary_churn_driver}
- **Suggested Offer to Weave into the Email:** {suggested_offer}

Now, please write the email based on all the rules and data provided.
"""

In [6]:
def get_prompt_inputs(customer_row):
    """Translates a customer's data profile into specific inputs for the LLM prompt."""
    profile = customer_row['loyalty_profile']
    tenure = customer_row['tenure']
    
    suggested_offer = ""
    # This is the logic layer connecting data insights to creative content
    if profile == "Freedom Seeker":
        suggested_offer = "A special loyalty discount on their monthly bill if they switch to a 12-month plan, giving them both savings and price stability."
    elif profile == "At-Risk VIP":
        suggested_offer = "A complimentary plan review with one of our experts to ensure they're on the best-value tier, plus a free month of our Premium Tech Support service as a thank you."
    elif profile == "New & Uncertain":
        suggested_offer = "A welcome gift of a free 3-month trial of our Premium Online Security package to help them get the most out of their new services and stay protected."
    else: # Fallback for "General Churn Risk"
        suggested_offer = "A special 'thank you' offer, giving them a complimentary speed boost on their internet for the next three months."
        
    return {
        "customer_name": np.random.choice(["Alex", "Jamie", "Katie","Ashelyn", "Vachan","Ben"]), # Randomly select a name for Demo
        "tenure_months": tenure,
        "loyalty_profile": profile,
        "primary_churn_driver": customer_row['primary_driver'],
        "suggested_offer": suggested_offer
    }

def generate_retention_email(customer_data):
    """Generates a personalized retention email for a given customer."""
    try:
        model = genai.GenerativeModel(llm_model)
        prompt_inputs = get_prompt_inputs(customer_data)
        final_prompt = MASTER_PROMPT.format(**prompt_inputs)
        
        response = model.generate_content(final_prompt)
        return response.text
    except Exception as e:
        return f"An error occurred while generating the email: {e}"


Example

In [7]:
number_of_examples = 1
# We'll pick a few unique high-risk profiles to demonstrate personalization.
if not high_risk_customers.empty:
    customers_to_showcase = high_risk_customers.drop_duplicates(subset=['loyalty_profile']).sample(n=number_of_examples)

    for index, customer in customers_to_showcase.iterrows():
        print("\n" + "="*80)
        print(f"GENERATING EMAIL FOR PROFILE: '{customer['loyalty_profile']}' (Churn Prob: {customer['churn_probability']:.2f})")
        print("="*80)
        
        generated_email = generate_retention_email(customer)
        display(Markdown(generated_email))

else:
    print("\nNo high-risk customers to generate emails for.")


GENERATING EMAIL FOR PROFILE: 'New & Uncertain' (Churn Prob: 0.57)


Subject: Enhancing Your Vodafone Experience, Alex!

Hi Alex,

We're reaching out to show our appreciation for you being a Vodafone customer for the past 8 months. We're always looking for ways to enhance your experience and want to share something special with you.

As a valued customer, we'd like to offer you a welcome gift: a free 3-month trial of our Premium Online Security package.  This added layer of protection will help keep you safe online while you enjoy all that Vodafone has to offer.

Here's what you'll get with our Premium Online Security package:

* **Enhanced protection against viruses, malware, and phishing attempts.**
* **Secure browsing for safer online shopping and banking.**
* **Protection for multiple devices.**

Click here to explore your exclusive benefits and activate your free 3-month trial.

We appreciate your choice to connect with Vodafone. We’re committed to providing you with the best possible service.

Best regards,
The Vodafone Customer Care Team 
